In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import string, nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
df=pd.read_csv("Reviews.csv")

In [4]:
df = df.head(50000)

In [6]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [9]:
df.isnull().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               3
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   2
Text                      0
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Id                      50000 non-null  int64 
 1   ProductId               50000 non-null  object
 2   UserId                  50000 non-null  object
 3   ProfileName             49997 non-null  object
 4   HelpfulnessNumerator    50000 non-null  int64 
 5   HelpfulnessDenominator  50000 non-null  int64 
 6   Score                   50000 non-null  int64 
 7   Time                    50000 non-null  int64 
 8   Summary                 49998 non-null  object
 9   Text                    50000 non-null  object
dtypes: int64(5), object(5)
memory usage: 3.8+ MB


In [11]:
df.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04
mean,25000.500000,1.603360,2.060260,4.145840,1.295198e+09
std,14433.901067,5.620771,6.216044,1.325596,4.734622e+07
min,1.000000,0.000000,0.000000,1.000000,9.617184e+08
25%,12500.750000,0.000000,0.000000,4.000000,1.269648e+09
50%,25000.500000,0.000000,1.000000,5.000000,1.309133e+09
75%,37500.250000,2.000000,2.000000,5.000000,1.331078e+09
max,50000.000000,398.000000,401.000000,5.000000,1.351210e+09


In [12]:
df['Score'].value_counts()

5    31130
4     7288
1     4721
3     4047
2     2814
Name: Score, dtype: int64

In [13]:
def clean_text(text):
    nopunc = [w for w in text if w not in string.punctuation]
    nopunc = ''.join(nopunc)
    return  ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

In [15]:
df['Text'][0], clean_text(df['Text'][0])

('I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.',
 'bought several Vitality canned dog food products found good quality product looks like stew processed meat smells better Labrador finicky appreciates product better')

In [16]:
df['Text'].head().apply(clean_text)

0    bought several Vitality canned dog food produc...
1    Product arrived labeled Jumbo Salted Peanutsth...
2    confection around centuries light pillowy citr...
3    looking secret ingredient Robitussin believe f...
4    Great taffy great price wide assortment yummy ...
Name: Text, dtype: object

In [17]:
df.shape

(50000, 10)

In [19]:
df['Text'] = df['Text'].apply(clean_text)

In [21]:
df['Text'] = df['Text'].astype(str)

In [22]:
def preprocess(text):
    return ' '.join([word for word in word_tokenize(text) if word not in stopwords.words('english') and not word.isdigit() and word not in string.punctuation])

In [23]:
preprocess(df['Text'][4])

'Great taffy great price wide assortment yummy taffy Delivery quick taffy lover deal'

In [25]:
df['Text'][:10000] = df['Text'][:10000].apply(preprocess)

In [26]:
df['Text'][10001:20000] = df['Text'][10001:20000].apply(preprocess)

In [27]:
df['Text'][20001:30000] = df['Text'][20001:30000].apply(preprocess)

In [28]:
df['Text'][30001:40000] = df['Text'][30001:40000].apply(preprocess)

In [29]:
df['Text'][40001:50000] = df['Text'][40001:50000].apply(preprocess)

In [30]:
df['Text'] = df['Text'].str.lower()

In [32]:
stemmer = PorterStemmer()
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])
df['Text'] = df['Text'].apply(lambda x: stem_words(x))

In [33]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
df["Text"] = df["Text"].apply(lambda text: lemmatize_words(text))

In [34]:
df['Text'].head()

0    bought sever vital can dog food product found ...
1    product arriv label jumbo salt peanutsth peanu...
2    confect around centuri light pillowi citru gel...
3    look secret ingredi robitussin believ found go...
4    great taffi great price wide assort yummi taff...
Name: Text, dtype: object

In [36]:
df.to_csv('Preprocessed Unlabeled Reviews Dataset.csv')